In [ ]:
from os import path, makedirs, rename, remove
from time import time
import argparse
import pickle

import numpy as np
import pandas as pd
import tensorflow_hub as hub
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer

In [ ]:
train_data = pd.read_csv('./data_fake_news/train.csv')
test_data = pd.read_csv('./data_fake_news/test.csv')

In [ ]:
test_data.head()

In [ ]:
descriptions = train_data['text']
train_size = int(len(descriptions) * .8)
labels = train_data['label']
train_description = descriptions[:train_size].astype('str')
train_labels = labels[:train_size]
test_description = descriptions[train_size:].astype('str')
test_labels = labels[train_size:]

In [ ]:
top_labels = ['0', '1']
description_embeddings = hub.text_embedding_column(
  "text", 
  module_spec="https://tfhub.dev/google/universal-sentence-encoder/2"
)

In [ ]:
encoder = LabelBinarizer()
encoder.fit_transform(train_labels)
train_encoded = encoder.transform(train_labels)
test_encoded = encoder.transform(test_labels)
num_classes = len(encoder.classes_)

In [ ]:
binary_label_head = tf.contrib.estimator.binary_classification_head(
                        loss_reduction=tf.losses.Reduction.SUM_OVER_BATCH_SIZE
                    )

In [ ]:
estimator = tf.contrib.estimator.DNNEstimator(
    head=binary_label_head,
    hidden_units=[64,10,100],
    feature_columns=[description_embeddings]
)

In [ ]:
estimator_linear_combined = tf.contrib.estimator.DNNLinearCombinedEstimator(
    head=binary_label_head,
    dnn_hidden_units=[64,10],
    dnn_feature_columns=[description_embeddings]
)

In [ ]:
estimator_linear = tf.contrib.estimator.LinearEstimator(
    head=binary_label_head,
    feature_columns=[description_embeddings]
)

In [ ]:
features = {
  "text": np.array(train_description).astype(np.str)
}
labels = np.array(train_encoded).astype(np.int32)

train_input_fn = tf.estimator.inputs.numpy_input_fn(
    features, 
    labels, 
    shuffle=True, 
    batch_size=32, 
    num_epochs=20
)

In [ ]:
tf.logging.set_verbosity(tf.logging.INFO)
estimator.train(input_fn=train_input_fn)

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn({"text": np.array(test_description).astype(np.str)}, test_encoded.astype(np.int32), shuffle=False)

estimator.evaluate(input_fn=eval_input_fn)

In [ ]:
raw_test = [
    "The BBC produced spoof on the â€œReal Housewivesâ€ TV programmes, which has a comedic Islamic State twist, has been criticised by Leftists and Muslims who claim the sketch is offensive. [The BBC released the trailer earlier this week and were immediately slammed by those on the left and Muslims who thought that making fun of the brides of members of the terror group was out of the bounds of conventional humour. The sketch is part of a new programme called â€œRevoltingâ€ written by Jolyon Rubinstein and Heydon Prowse which, according to the BBC, is â€œsatirising the state of the nation. â€  LOLOLOL pic. twitter.   â€”   Raheem Kassam (@RaheemKassam) January 4, 2017,  The BBC2 Facebook page was inundated with criticism. One user wrote, â€œIâ€™m mortified that the BBC had produced such a programme. This is simply bad taste. The fact it is a comedy makes it even more worrying that humour should be associated with the actions of ISIS. Is this really what TV licenses are funding! ?â€ Others took a different point of view including a Muslim saying, â€œAs a Muslim I find this HILARIOUS! Brilliant! The satire is on point highlighting the pathetic ideals of a pathetic group like ISIS. â€ Not all Muslims agreed with the sentiment and users on Twitter expressed just as much outrage for the sketch. One man, a Bangladeshi, said â€œThe BBC really made a satirical show called â€˜The Real Housewives of ISISâ€™ while the real housewives of ISIS are being raped and abused daily. â€   The BBC really made a satirical show called â€The Real Housewives of ISISâ€ while the real housewives of ISIS are being raped and abused daily,  â€”   Meraj. (@UncleMeraj) January 4, 2017,  Another wrote, â€œâ€˜Real Housewives of ISISâ€™ will make Hijabis feel more isolated n targeted by Islamophobes. Thanks @BBC for adding to the negative stereotype. â€ â€™Real Housewives of ISISâ€™ will make Hijabis feel more isolated n targeted by Islamophobes. Thanks @BBC for adding to the negative stereotype,  â€”   aÑ•Ð½ (@AshKaneSkittles) January 4, 2017,  Leftists also articulated how offended they were that the public broadcaster would dare create such a sketch mocking Islamic State. Some questioned whether the use of taxpayer money via the TV license fee should go toward the funding of the programme. As if peopleâ€™s TV license in this country is going towards funding the production of a programme called â€the real housewives of ISISâ€. Wow,  â€”   Cameron Edgar (@CammyyyEdgar) January 4, 2017,    The video itself has already been viewed millions of times on Facebook and other social media platforms.  The clip shows several women in hijabs talking to each other in a    house taking selfies and showing off their suicide belts to each other. One woman even mentions that she hadnâ€™t come from Birmingham â€œto do thisâ€ as she scrubbed the floor of the home. Women joining Islamic State and travelling to Syria to become brides has become a real problem in European countries as the terror group promises young girls a more glamorous lifestyle. In 2014, eight schoolgirls from Bethnal Green travelled to Syria to become brides of Islamic State fighters, all of them under the age of 18."
]

In [ ]:
predict_input_fn = tf.estimator.inputs.numpy_input_fn({"text": np.array(raw_test).astype(np.str)}, shuffle=False)
results = estimator.predict(predict_input_fn)

In [ ]:
print(list(results)[0])